# Install dependencies

In [1]:
!pip install requests==2.23.0 numpy==1.18.5 pandas==1.0.3 \
    scikit-learn==0.23.1 pytorch-lightning==0.8.4 torch==1.5.1 \
    transformers==3.0.2 sklearn==0.0 tqdm==4.45.0 neptune-client==0.4.119 \
    matplotlib==3.1.0 scikit-plot==0.3.7

  Using cached requests-2.23.0-py2.py3-none-any.whl.metadata (6.8 kB)
  Using cached numpy-1.18.5.zip (5.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [26 lines of output]
  Running from numpy source directory.
  <string>:461: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
  Traceback (most recent call last):
    File "E:\Python\Python3.11.5\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
      main()
    File "E:\Python\Python3.11.5\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
      json_out["return_val"] = hook(**hook_input["kwargs"])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "E:\Python\Python3.11.5\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 175, in prepare_metadata_for_build_wheel
      return hook(metadata_directory, config_settings)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [1]:
!pip install pytorch-lightning
!pip install neptune

# Import needed modules

In [3]:
import gc
import json
import math
import os
import warnings
from abc import ABC, abstractmethod
from collections import OrderedDict
from random import randint
from typing import Iterable, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from matplotlib.figure import Figure
from pandas import DataFrame
from pytorch_lightning import LightningModule, seed_everything
from pytorch_lightning import Trainer as LightningTrainer
from pytorch_lightning.loggers import NeptuneLogger
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.utils import column_or_1d
from torch import Tensor, nn
from torch.nn import functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, IterableDataset
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForSequenceClassification,
    GenerationMixin, TFGenerationMixin
)

# 忽略 transformers 可能的 FutureWarning
warnings.simplefilter("ignore", category=FutureWarning)

# Define constants

In [4]:
# --- Random seed ---
SEED = 2020
seed_everything(SEED)

# --- Directory ---
ROOT_DIR = os.path.abspath(".")
PROCESSED_DATA_DIR = os.path.join(ROOT_DIR, "data/processed") 
METADATA_FILE_NAME = os.path.join(PROCESSED_DATA_DIR, "metadata.json")
CHECKPOINT_DIR = os.path.join(ROOT_DIR, "checkpoint")

KAGGLE_ENV = bool(os.getenv("KAGGLE_URL_BASE"))
if KAGGLE_ENV:
    # in Kaggle environment
    # 2 datasets should already been added to the notebook
    RAW_DATA_DIR = os.path.join(ROOT_DIR, "../input")
else:
    # in local environment
    RAW_DATA_DIR =  os.path.join(ROOT_DIR, "data/raw")

# --- Datasets ---
DATASET_MAPPING = {
    "SemEval2010Task8": {
        "dir": os.path.join(RAW_DATA_DIR,"semeval2010-task-8"),
        "keep_test_order": True,
        "precision_recall_curve_baseline_img": None,
    },
    "GIDS": {
        "dir": os.path.join(RAW_DATA_DIR,"gids-dataset"),
        "keep_test_order": False,
        "precision_recall_curve_baseline_img": os.path.join(RAW_DATA_DIR,"gids-dataset/GIDS_precision_recall_curve.png"),
    }
}

# change this variable to switch dataset in later tasks
DATASET_NAME = list(DATASET_MAPPING.keys())[1]

# --- Subject & object markup ---
SUB_START_CHAR = "["
SUB_END_CHAR = "]"
OBJ_START_CHAR = "{"
OBJ_END_CHAR = "}"

# --- BERT variants ---
# See https://huggingface.co/transformers/pretrained_models.html for the full list
AVAILABLE_PRETRAINED_MODELS = [
    "distilbert-base-uncased", # 0
    "distilbert-base-cased",   # 1
    "bert-base-uncased",       # 2
    "distilgpt2",              # 3
    "gpt2",                    # 4
    "distilroberta-base",      # 5
    "roberta-base",            # 6
    "albert-base-v1",          # 7
    "albert-base-v2",          # 8
    "bert-large-uncased",      # 9
]

# change this variable to switch pretrained language model
PRETRAINED_MODEL = AVAILABLE_PRETRAINED_MODELS[2]

# if e1 is not related to e2, should "e2 not related to e1" be added to the training set
ADD_REVERSE_RELATIONSHIP = True

# --- Neptune logger ---
# Create a free account at https://neptune.ai/,
# then get the API token and create a project
NEPTUNE_API_TOKEN = " eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0YmEyZTg2ZC0zNDMxLTQ2MzItOWEzYS0xOWY5ZGNiYTA5YWYifQ== "
NEPTUNE_PROJECT_NAME = " Violetta/BERT "

Seed set to 2020


# Get Dataset

In [10]:
import os
import requests
from datasets import load_dataset

# 定义数据集存储路径
DATASET_MAPPING = {
    "SemEval2010Task8": {
        "dir": "datasets/semeval2010"
    },
    "GIDS": {
        "dir": "datasets/gids"
    }
}

# GIDS 数据集的下载 URL
GIDS_URLs = {
    "train": "https://raw.githubusercontent.com/SharmisthaJat/RE-DS-Word-Attention-Models/master/Data/GIDS/train.tsv",
    "validation": "https://raw.githubusercontent.com/SharmisthaJat/RE-DS-Word-Attention-Models/master/Data/GIDS/dev.tsv",
    "test": "https://raw.githubusercontent.com/SharmisthaJat/RE-DS-Word-Attention-Models/master/Data/GIDS/test.tsv",
}

# 确保所有数据集目录存在
for dataset, paths in DATASET_MAPPING.items():
    os.makedirs(paths["dir"], exist_ok=True)

# 下载 GIDS 数据集
def download_gids():
    gids_dir = DATASET_MAPPING["GIDS"]["dir"]

    for split, url in GIDS_URLs.items():
        file_name = "val.tsv" if split == "validation" else f"{split}.tsv"
        file_path = os.path.join(gids_dir, file_name)

        if not os.path.exists(file_path):
            print(f"Downloading GIDS {split} set from {url}...")
            response = requests.get(url, stream=True)

            if response.status_code == 200:
                with open(file_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                print(f"{file_name} downloaded successfully.")
            else:
                print(f"Failed to download {file_name}. Please check the URL.")
        else:
            print(f"{file_name} already exists. Skipping download.")

# 下载并保存 SemEval 2010 Task 8 数据集
def download_semeval():
    semeval_dir = DATASET_MAPPING["SemEval2010Task8"]["dir"]
    train_dir = os.path.join(semeval_dir, "SemEval2010_task8_training")
    test_dir = os.path.join(semeval_dir, "SemEval2010_task8_testing_keys")

    train_path = os.path.join(train_dir, "TRAIN_FILE.TXT")
    test_path = os.path.join(test_dir, "TEST_FILE_FULL.TXT")

    if not os.path.exists(train_path) or not os.path.exists(test_path):
        print("Downloading SemEval2010 Task 8 dataset from Hugging Face...")
        dataset = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

        # **检查数据集是否成功加载**
        print("Dataset structure:", dataset)
        print("Train set size:", len(dataset["train"]))
        print("Test set size:", len(dataset["test"]))

        # 确保目录存在
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        # **检查数据格式**
        for example in dataset["train"][:5]:  
            print(f"DEBUG: Example: {example}")

        # **保存训练集**
        with open(train_path, "w", encoding="utf-8") as f:
            for example in dataset["train"]:
                if not isinstance(example, dict):  
                    print(f"Unexpected data format in train set: {example}")
                    continue  # 跳过格式错误的样本
                
                sentence = example.get("sentence", "").strip()
                relation = str(example.get("relation", "Other")).strip()
                
                if not sentence or not relation:
                    print(f"Skipping empty entry: {example}")
                    continue  # 跳过空值数据
                
                f.write(f"{sentence}\t{relation}\n")
        print(f"Training set saved to {train_path}")

        # **保存测试集**
        with open(test_path, "w", encoding="utf-8") as f:
            for example in dataset["test"]:
                if not isinstance(example, dict):  
                    print(f"Unexpected data format in test set: {example}")
                    continue  # 跳过格式错误的样本

                sentence = example.get("sentence", "").strip()
                relation = str(example.get("relation", "Other")).strip()

                if not sentence or not relation:
                    print(f"Skipping empty entry: {example}")
                    continue  # 跳过空值数据

                f.write(f"{sentence}\t{relation}\n")
        print(f"Test set saved to {test_path}")

    else:
        print("SemEval2010 Task 8 dataset already exists.")

# 执行数据下载
download_gids()
download_semeval()

# 验证数据集路径
print("\nDataset storage locations:")
print(f"SemEval 2010 Task 8 directory: {DATASET_MAPPING['SemEval2010Task8']['dir']}")
print(f"GIDS directory: {DATASET_MAPPING['GIDS']['dir']}")

train.tsv downloaded successfully.
val.tsv downloaded successfully.
test.tsv downloaded successfully.
Dataset structure: DatasetDict({
    train: Dataset({
        features: ['sentence', 'relation'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence', 'relation'],
        num_rows: 2717
    })
})
Train set size: 8000
Test set size: 2717
DEBUG: Example: sentence
DEBUG: Example: relation
Training set saved to datasets/semeval2010\SemEval2010_task8_training\TRAIN_FILE.TXT
Test set saved to datasets/semeval2010\SemEval2010_task8_testing_keys\TEST_FILE_FULL.TXT

Dataset storage locations:
SemEval 2010 Task 8 directory: datasets/semeval2010
GIDS directory: datasets/gids


# Preprocess

In [13]:
class OrdinalLabelEncoder:
    def __init__(self, init_labels=None):
        if init_labels is None:
            init_labels = []
        self.mapping = OrderedDict({l: i for i, l in enumerate(init_labels)})

    @property
    def classes_(self):
        return list(self.mapping.keys())

    def fit_transform(self, y):
        return self.fit(y).transform(y)

    def fit(self, y):
        y = column_or_1d(y, warn=True)
        new_classes = pd.Series(y).unique()
        for cls in new_classes:
            if cls not in self.mapping:
                self.mapping[cls] = len(self.mapping)
        return self

    def transform(self, y):
        y = column_or_1d(y, warn=True)
        return [self.mapping[value] for value in y]

## Abstract preprocessor class:

In [14]:
from transformers import PreTrainedTokenizer

class AbstractPreprocessor(ABC):
    DATASET_NAME = ""
    VAL_DATA_PROPORTION = 0.2
    NO_RELATION_LABEL = ""

    def __init__(self, tokenizer: PreTrainedTokenizer):
        self.tokenizer = tokenizer
        self.SUB_START_ID, self.SUB_END_ID, self.OBJ_START_ID, self.OBJ_END_ID \
            = tokenizer.convert_tokens_to_ids([SUB_START_CHAR, SUB_END_CHAR, OBJ_START_CHAR, OBJ_END_CHAR])
        self.label_encoder = OrdinalLabelEncoder([self.NO_RELATION_LABEL])

    def preprocess_data(self, reprocess: bool):
        print(f"\n---> Preprocessing {self.DATASET_NAME} dataset <---")
        
        # create processed data dir
        if not os.path.exists(PROCESSED_DATA_DIR):
            print("Creating processed data directory " + PROCESSED_DATA_DIR)
            os.makedirs(PROCESSED_DATA_DIR)

        # stop preprocessing if file existed
        json_file_names = [self.get_dataset_file_name(k) for k in ("train", "val", "test")]
        existed_files = [fn for fn in json_file_names if os.path.exists(fn)]
        if existed_files:
            file_text = "- " + "\n- ".join(existed_files)
            if not reprocess:
                print("The following files already exist:")
                print(file_text)
                print("Preprocessing is skipped. See option --reprocess.")
                return
            else:
                print("The following files will be overwritten:")
                print(file_text)

        train_data, val_data, test_data = self._preprocess_data()

        print("Saving to json files")
        self._write_data_to_file(train_data, "train")
        self._write_data_to_file(val_data, "val")
        self._write_data_to_file(test_data, "test")

        self._save_metadata({
            "train_size": len(train_data),
            "val_size": len(val_data),
            "test_size": len(test_data),
            "no_relation_label": self.NO_RELATION_LABEL,
            **self._get_label_mapping()
        })

        self._create_secondary_data_files()

        print("---> Done ! <---")

    @abstractmethod
    def _preprocess_data(self) -> Tuple[DataFrame, DataFrame, DataFrame]:
        pass

    def _create_secondary_data_files(self):
        """
        From the primary data file, create a data file with binary labels
        and a data file with only sentences classified as "related"
        """

        with open(METADATA_FILE_NAME) as f:
            root_metadata = json.load(f)
            metadata = root_metadata[self.DATASET_NAME]

        related_only_count = {
            "train": 0,
            "val": 0,
            "test": 0,
        }

        for key in ["train", "test", "val"]:
            print(f"Creating secondary files for {key} data")

            origin_file = open(self.get_dataset_file_name(key))
            bin_file = open(self.get_dataset_file_name(f"{key}_binary"), "w")
            related_file = open(self.get_dataset_file_name(f"{key}_related_only"), "w")

            total = metadata[f"{key}_size"]

            for line in tqdm(origin_file, total=total):
                data = json.loads(line)
                if data["label"] != 0:
                    related_only_count[key] += 1
                    data["label"] -= 1 # label in "related_only" files is 1 less than the original label
                    related_file.write(json.dumps(data) + "\n")
                    data["label"] = 1 # in binary dataset, all "related" classes have label 1
                    bin_file.write(json.dumps(data) + "\n")
                else:
                    bin_file.write(json.dumps(data) + "\n")

            origin_file.close()
            bin_file.close()
            related_file.close()

        print("Updating metadata.json")
        for key in ["train", "test", "val"]:
            metadata[f"{key}_related_only_size"] = related_only_count[key]
        root_metadata[self.DATASET_NAME] = metadata
        with open(METADATA_FILE_NAME, "w") as f:
            json.dump(root_metadata, f, indent=4)

    def _find_sub_obj_pos(self, input_ids_list: Iterable) -> DataFrame:
        """
        Find subject and object position in a sentence
        """
        sub_start_pos = [self._index(s, self.SUB_START_ID) + 1 for s in input_ids_list]
        sub_end_pos = [self._index(s, self.SUB_END_ID, sub_start_pos[i]) for i, s in enumerate(input_ids_list)]
        obj_start_pos = [self._index(s, self.OBJ_START_ID) + 1 for s in input_ids_list]
        obj_end_pos = [self._index(s, self.OBJ_END_ID, obj_start_pos[i]) for i, s in enumerate(input_ids_list)]
        return DataFrame({
            "sub_start_pos": sub_start_pos,
            "sub_end_pos": sub_end_pos,
            "obj_start_pos": obj_start_pos,
            "obj_end_pos": obj_end_pos,
        })

    @staticmethod
    def _index(lst: list, ele: int, start: int = 0) -> int:
        """
        Find an element in a list. Returns -1 if not found instead of raising an exception.
        """
        try:
            return lst.index(ele, start)
        except ValueError:
            return -1

    def _clean_data(self, raw_sentences: list, labels: list) -> DataFrame:
        if not raw_sentences:
            return DataFrame()

        tokens = self.tokenizer(raw_sentences, truncation=True, padding="max_length")
        data = DataFrame(tokens.data)
        data["label"] = self.label_encoder.fit_transform(labels)
        sub_obj_position = self._find_sub_obj_pos(data["input_ids"])
        data = pd.concat([data, sub_obj_position], axis=1)
        data = self._remove_invalid_sentences(data)
        return data

    def _remove_invalid_sentences(self, data: DataFrame) -> DataFrame:
        """
        Remove sentences without subject/object or whose subject/object
        is beyond the maximum length the model supports
        """
        seq_max_len = self.tokenizer.model_max_length
        return data.loc[
            (data["sub_end_pos"] < seq_max_len)
            & (data["obj_end_pos"] < seq_max_len)
            & (data["sub_end_pos"] > -1)
            & (data["obj_end_pos"] > -1)
        ]

    def _get_label_mapping(self):
        """
        Returns a mapping from id to label and vise versa from the label encoder
        """
        # all labels
        id_to_label = dict(enumerate(self.label_encoder.classes_))
        label_to_id = {v: k for k, v in id_to_label.items()}

        # for the related_only dataset
        # ignore id 0, which represent no relation
        id_to_label_related_only = {k - 1: v for k, v in id_to_label.items() if k != 0}
        label_to_id_related_only = {v: k for k, v in id_to_label_related_only.items()}

        return {
            "id_to_label": id_to_label,
            "label_to_id": label_to_id,
            "id_to_label_related_only": id_to_label_related_only,
            "label_to_id_related_only": label_to_id_related_only,            
        }

    def _write_data_to_file(self, dataframe: DataFrame, subset: str):
        """Write data in a dataframe to train/val/test file"""
        lines = ""
        for _, row in dataframe.iterrows():
            lines += row.to_json() + "\n"
        with open(self.get_dataset_file_name(subset), "w") as file:
            file.write(lines)

    def _save_metadata(self, metadata: dict):
        """Save metadata to metadata.json"""
        # create metadata file
        if not os.path.exists(METADATA_FILE_NAME):
            print(f"Create metadata file at {METADATA_FILE_NAME}")
            with open(METADATA_FILE_NAME, "w") as f:
                f.write("{}\n")

        # add metadata
        print("Saving metadata")
        with open(METADATA_FILE_NAME) as f:
            root_metadata = json.load(f)
        with open(METADATA_FILE_NAME, "w") as f:
            root_metadata[self.DATASET_NAME] = metadata
            json.dump(root_metadata, f, indent=4)

    @classmethod
    def get_dataset_file_name(cls, key: str) -> str:
        return os.path.join(PROCESSED_DATA_DIR, f"{cls.DATASET_NAME.lower()}_{key}.json")

## Concrete preprocessor for each dataset:

In [15]:
class SemEval2010Task8Preprocessor(AbstractPreprocessor):
    DATASET_NAME = "SemEval2010Task8"
    NO_RELATION_LABEL = "Other"
    RAW_TRAIN_FILE_NAME = os.path.join(DATASET_MAPPING["SemEval2010Task8"]["dir"],
                                       "SemEval2010_task8_training/TRAIN_FILE.TXT")
    RAW_TEST_FILE_NAME = os.path.join(DATASET_MAPPING["SemEval2010Task8"]["dir"],
                                      "SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT")
    RAW_TRAIN_DATA_SIZE = 8000
    RAW_TEST_DATA_SIZE = 2717

    def _preprocess_data(self):
        print("Processing training data")
        train_data = self._process_file(
            self.RAW_TRAIN_FILE_NAME,
            self.RAW_TRAIN_DATA_SIZE,
            ADD_REVERSE_RELATIONSHIP,
        )

        print("Processing test data")
        test_data = self._process_file(
            self.RAW_TEST_FILE_NAME,
            self.RAW_TEST_DATA_SIZE,
            False,
        )

        print("Splitting train & validate data")
        train_data, val_data = train_test_split(train_data, shuffle=True, random_state=SEED)

        return train_data, val_data, test_data

    def _process_file(self, file_name: str, dataset_size: int, add_reverse: bool) -> DataFrame:
        raw_sentences = []
        labels = []
        with open(file_name) as f:
            for _ in tqdm(range(dataset_size)):
                sent = f.readline()
                label, sub, obj = self._process_label(f.readline())
                labels.append(label)
                raw_sentences.append(self._process_sentence(sent, sub, obj))
                if label == "Other" and add_reverse:
                    labels.append(label)
                    raw_sentences.append(self._process_sentence(sent, obj, sub))
                f.readline()
                f.readline()

        return self._clean_data(raw_sentences, labels)

    @staticmethod
    def _process_sentence(sentence: str, sub: int, obj: int) -> str:
        return sentence.split("\t")[1][1:-2] \
            .replace(f"<e{sub}>", SUB_START_CHAR) \
            .replace(f"</e{sub}>", SUB_END_CHAR) \
            .replace(f"<e{obj}>", OBJ_START_CHAR) \
            .replace(f"</e{obj}>", OBJ_END_CHAR)

    @staticmethod
    def _process_label(label: str) -> Tuple[str, int, int]:
        label = label.strip()
        if label == "Other":
            return label, 1, 2
        nums = list(filter(str.isdigit, label))
        return label, int(nums[0]), int(nums[1])


class GIDSPreprocessor(AbstractPreprocessor):
    DATASET_NAME = "GIDS"
    RAW_TRAIN_FILE_NAME = os.path.join(DATASET_MAPPING["GIDS"]["dir"], "train.tsv")
    RAW_VAL_FILE_NAME = os.path.join(DATASET_MAPPING["GIDS"]["dir"], "val.tsv")
    RAW_TEST_FILE_NAME = os.path.join(DATASET_MAPPING["GIDS"]["dir"], "test.tsv")
    TRAIN_SIZE = 11297
    VAL_SIZE = 1864
    TEST_SIZE = 5663
    NO_RELATION_LABEL = "NA"

    def _process_file(self, file_name: str, add_reverse: bool) -> DataFrame:
        """
        Process a file in batches
        Return the total data size
        """
        with open(file_name) as in_file:
            lines = in_file.readlines()
            
        raw_sentences = []
        labels = []

        for line in tqdm(lines):
            _, _, sub, obj, label, sent = line.split("\t")
            sent = sent.replace("###END###", "")

            # add subject markup
            new_sub = SUB_START_CHAR + " " + sub.replace("_", " ") + " " + SUB_END_CHAR
            new_obj = OBJ_START_CHAR + " " +  obj.replace("_", " ") + " " + OBJ_END_CHAR
            sent = sent.replace(sub, new_sub).replace(obj, new_obj)
            raw_sentences.append(sent)
            labels.append(label)

            if add_reverse and label == self.NO_RELATION_LABEL:
                new_sub = OBJ_START_CHAR + " " + sub.replace("_", " ") + " " + OBJ_END_CHAR
                new_obj = SUB_START_CHAR + " " + obj.replace("_", " ") + " " + SUB_END_CHAR
                sent = sent.replace(sub, new_sub).replace(obj, new_obj)
                raw_sentences.append(sent)
                labels.append(label)

        return self._clean_data(raw_sentences, labels)

    def _preprocess_data(self):
        
        print("Process train dataset")
        train_data = self._process_file(
            self.RAW_TRAIN_FILE_NAME,
            ADD_REVERSE_RELATIONSHIP,
        )

        print("Process val dataset")
        val_data = self._process_file(
            self.RAW_VAL_FILE_NAME,
            False,
        )
        
        print("Process test dataset")
        test_data = self._process_file(
            self.RAW_TEST_FILE_NAME,
            False,
        )

        return train_data, val_data, test_data

## Factory method to create preprocessors:

In [16]:
def get_preprocessor_class(dataset_name: str = DATASET_NAME):
    return globals()[f"{dataset_name}Preprocessor"]
        
def get_preprocessor(dataset_name: str = DATASET_NAME)-> AbstractPreprocessor:
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL, use_fast=True)
    # some tokenizer, like GPTTokenizer, doesn't have pad_token
    # in this case, we use eos token as pad token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    preprocessors_class = get_preprocessor_class(dataset_name)
    return preprocessors_class(tokenizer)

## Preprocess data:

In [17]:
preprocessor = get_preprocessor()
preprocessor.preprocess_data(reprocess=True)

loading configuration file config.json from cache at C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\vocab.txt
loa


---> Preprocessing GIDS dataset <---
Creating processed data directory E:\Python\TextMining\BERT\data/processed
Process train dataset


  0%|          | 0/11297 [00:00<?, ?it/s]

Process val dataset


  0%|          | 0/1864 [00:00<?, ?it/s]

...

Creating secondary files for test data


  0%|          | 0/5653 [00:00<?, ?it/s]

Creating secondary files for val data


  0%|          | 0/1864 [00:00<?, ?it/s]

Updating metadata.json
---> Done ! <---


# Dataset

In [19]:
class GenericDataset(IterableDataset):
    """A generic dataset for train/val/test data for both SemEval and GIDS dataset"""

    def __init__(self, dataset_name: str, subset: str, batch_size: int, label_transform: str):
        assert subset in ["train", "val", "test"]
        assert label_transform in ["none", "binary", "related_only"]

        file_name = subset if label_transform == "none" \
            else f"{subset}_{label_transform}"

        preprocessor_class = get_preprocessor_class()
        with open(METADATA_FILE_NAME) as f:
            metadata = json.load(f)[dataset_name]

        size = metadata[f"{subset}_related_only_size"] \
            if label_transform == "related_only" \
            else metadata[f"{subset}_size"]

        self.subset = subset
        self.batch_size = batch_size
        self.length = math.ceil(size / batch_size)
        self.file = open(preprocessor_class.get_dataset_file_name(file_name))

        self.keep_test_order = self.subset == "test" and DATASET_MAPPING[dataset_name]["keep_test_order"]

    def __del__(self):
        if self.file:
            self.file.close()

    def __iter__(self):
        """
        Implement "smart batching"
        """

        data = [json.loads(line) for line in self.file]
        if not self.keep_test_order:
            data = sorted(data, key=lambda x: sum(x["attention_mask"]))

        new_data = []

        while len(data) > 0:
            if self.keep_test_order or len(data) < self.batch_size:
                idx = 0
            else:
                idx = randint(0, len(data) - self.batch_size)
            batch = data[idx:idx + self.batch_size]
            max_len = max([sum(b["attention_mask"]) for b in batch])

            for b in batch:
                input_data = {}
                for k, v in b.items():
                    if k != "label":
                        if isinstance(v, list):
                            input_data[k] = torch.tensor(v[:max_len])
                        else:
                            input_data[k] = torch.tensor(v)
                label = torch.tensor(b["label"])
                new_data.append((input_data, label))

            del data[idx:idx + self.batch_size]

        yield from new_data
    
    def __len__(self):
        return self.length

    def as_batches(self):
        input_data = []
        label = []
        
        def create_batch():
            return (
                {k: torch.stack([x[k] for x in input_data]).cuda() for k in input_data[0].keys()},
                torch.tensor(label).cuda()
            )
        
        for ip, l in self:
            input_data.append(ip)
            label.append(l)
            if len(input_data) == self.batch_size:
                yield create_batch()
                input_data.clear()
                label.clear()

        yield create_batch()

# Classifiers

In [20]:
class BaseClassifier(LightningModule, ABC):
    """
    Base class of all classifiers
    """
    
    dataset_label_transform = None

    @abstractmethod
    def loss_function(self, logits: Tensor, label: Tensor) -> Tensor:
        """
        Calculate the loss of the model
        It MUST take care of the last activation layer
        """
        pass
    
    @abstractmethod
    def log_metrics(self, epoch_type: str, logits: Tensor, label: Tensor) -> dict:
        pass

    def __init__(self, pretrained_language_model, dataset_name, batch_size, learning_rate, decay_lr_speed,
                 dropout_p, activation_function, weight_decay, linear_size):
        super().__init__()
        self.save_hyperparameters()
        self.test_proposed_answer = None

        self.language_model = AutoModel.from_pretrained(pretrained_language_model)
        config = self.language_model.config
        self.max_seq_len = config.max_position_embeddings
        self.hidden_size = config.hidden_size

        self.linear = nn.Linear(self.hidden_size, linear_size)
        self.linear_output = nn.Linear(linear_size, self.num_classes)

        self.dropout = nn.Dropout(p=dropout_p)
        self.activation_function = getattr(nn, activation_function)()

    def forward(self, sub_start_pos, sub_end_pos,
                obj_start_pos, obj_end_pos, *args, **kwargs) -> Tensor:
        language_model_output = self.language_model(*args, **kwargs)
        if isinstance(language_model_output, tuple):
            language_model_output = language_model_output[0]

        x = torch.mean(language_model_output, dim=1)
        x = self.dropout(x)
        x = self.linear(x)
        x = self.activation_function(x)
        x = self.dropout(x)
        logits = self.linear_output(x)

        return logits

    def train_dataloader(self) -> DataLoader:
        return self.__get_dataloader("train")

    def val_dataloader(self) -> DataLoader:
        return self.__get_dataloader("val")

    def test_dataloader(self) -> DataLoader:
        return self.__get_dataloader("test")

    def __get_dataloader(self, subset: str) -> DataLoader:
        batch_size = self.hparams.batch_size
        dataset = GenericDataset(
            self.hparams.dataset_name,
            subset, 
            batch_size, 
            self.dataset_label_transform
        )
        return DataLoader(
            dataset,
            batch_size=batch_size,
            num_workers=1
        )

    def configure_optimizers(self):
        optimizer = AdamW(
            [p for p in self.parameters() if p.requires_grad],
            lr=self.hparams.learning_rate,
            weight_decay=self.hparams.weight_decay
        )
        scheduler = LambdaLR(optimizer, lambda epoch: self.hparams.decay_lr_speed[epoch])
        return [optimizer], [scheduler]
    
    def training_step(self, batch: Tuple[dict, Tensor], batch_nb: int) -> dict:
        input_data, label = batch
        logits = self(**input_data)

        loss = self.loss_function(logits, label)
        log = {"train_loss": loss}

        return {"loss": loss, "log": log}

    def __eval_step(self, batch:  Tuple[dict, Tensor]) -> dict:
        input_data, label = batch
        logits = self(**input_data)

        return {
            "logits": logits,
            "label": label,
        }
    
    def validation_step(self, batch: Tuple[dict, Tensor], batch_nb: int) -> dict:
        return self.__eval_step(batch)
    
    def test_step(self, batch: Tuple[dict, Tensor], batch_nb: int) -> dict:
        return self.__eval_step(batch)

    def __eval_epoch_end(self, epoch_type: str, outputs: Iterable[dict]) -> dict:
        assert epoch_type in ["test", "val"]
        
        logits = torch.cat([x["logits"] for x in outputs]).cpu()
        label = torch.cat([x["label"] for x in outputs]).cpu()
        
        logs = self.log_metrics(epoch_type, logits, label)
        
        return {"progress_bar": logs}
    
    def validation_epoch_end(self, outputs: Iterable[dict]) -> dict:
        return self.__eval_epoch_end("val", outputs)

    def test_epoch_end(self, outputs: Iterable[dict]) -> dict:
        return self.__eval_epoch_end("test", outputs)
    
    def numeric_labels_to_text(self, label):
        """Revert labels from number to text"""
        if self.dataset_label_transform == "binary":
            label = ["Positive" if x else "Negative" for x in label]
        else:
            with open(METADATA_FILE_NAME) as f:
                meta = json.load(f)[self.hparams.dataset_name]
            if self.dataset_label_transform == "none":
                mapping = meta["id_to_label"]
            else:
                mapping = meta["id_to_label_related_only"]
            label = [mapping[str(int(x))] for x in label]
        return label

    @staticmethod
    def plot_confusion_matrix(predicted_label, label) -> Figure:
        result = confusion_matrix(label, predicted_label)
        display = ConfusionMatrixDisplay(result)
        fig, ax = plt.subplots(figsize=(16, 12))
        display.plot(cmap=plt.cm.get_cmap("Blues"), ax=ax, xticks_rotation='vertical')
        return fig

    def log_confusion_matrix(self, prefix: str, predicted_label: Tensor, label: Tensor):
        predicted_label = self.numeric_labels_to_text(predicted_label)
        label = self.numeric_labels_to_text(label)
        fig = self.plot_confusion_matrix(predicted_label, label)
        self.logger.experiment.log_image(f"{prefix}_confusion_matrix", fig)


class MulticlassClassifier(BaseClassifier, ABC):
    """
    Base class for multiclass classifiers
    """

    def loss_function(self, logits: Tensor, label: Tensor)-> Tensor:
        return F.cross_entropy(logits, label)

    @staticmethod
    def logits_to_label(logits: Tensor) -> Tensor:
        return torch.argmax(logits, dim=-1)

    def log_metrics(self, epoch_type: str, logits: Tensor, label: Tensor) -> dict:
        predicted_label = self.logits_to_label(logits)
        self.log_confusion_matrix(epoch_type, predicted_label, label)

        logs = {
            f"{epoch_type}_avg_loss": float(self.loss_function(logits, label)),
            f"{epoch_type}_acc": accuracy_score(label, predicted_label),
            f"{epoch_type}_pre_weighted": precision_score(label, predicted_label, average="weighted"),
            f"{epoch_type}_rec_weighted": recall_score(label, predicted_label, average="weighted"),
            f"{epoch_type}_f1_weighted": f1_score(label, predicted_label, average="weighted"),
            f"{epoch_type}_pre_macro": precision_score(label, predicted_label, average="macro"),
            f"{epoch_type}_rec_macro": recall_score(label, predicted_label, average="macro"),
            f"{epoch_type}_f1_macro": f1_score(label, predicted_label, average="macro"),
        }

        for k, v in logs.items():
            self.logger.experiment.log_metric(k, v)

        return logs


class StandardClassifier(MulticlassClassifier):
    """
    A classifier that can recognize the "not related" as well as other relations
    """

    dataset_label_transform = "none"

    def __init__(self, dataset_name, **kwargs):
        with open(METADATA_FILE_NAME) as f:
            self.num_classes = len(json.load(f)[dataset_name]["label_to_id"])
        self.test_proposed_answer = None
        super().__init__(dataset_name=dataset_name, **kwargs)


    def log_metrics(self, epoch_type: str, logits: Tensor, label: Tensor)-> dict:
        if epoch_type == "test":
            self.test_proposed_answer = self.logits_to_label(logits).tolist()
        self.__log_precision_recall_curve(epoch_type, logits, label)
        return super().log_metrics(epoch_type, logits, label)

    def __log_precision_recall_curve(self, epoch_type: str, logits: Tensor, label: Tensor):
        """
        Log the micro-averaged precision recall curve
        Ref: https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
        """

        label = torch.tensor(label_binarize(label, classes=list(range(self.num_classes)))).flatten()
        logits = logits.flatten()

        pre, rec, thresholds = precision_recall_curve(label, logits)
        f1s = 2 * pre * rec / (pre + rec)
        ix = np.argmax(f1s)

        fig, ax = plt.subplots(figsize=(10, 10))

        # render the baseline curves as background for comparison
        background = DATASET_MAPPING[self.hparams.dataset_name]["precision_recall_curve_baseline_img"]
        if background:
            img = plt.imread(background)
            ax.imshow(img, extent=[0, 1, 0, 1])

        no_skill = len(label[label == 1]) / len(label)
        ax.plot(rec, pre, label="Our proposed model", color="blue")
        ax.set_xlabel("Recall")
        ax.set_ylabel("Precision")
        ax.legend()

        self.logger.experiment.log_image(f"{epoch_type}_pre_rec_curve", fig)
        self.logger.experiment.log_metric(
            f"{epoch_type}_average_precision_score_micro",
            average_precision_score(label, logits, average="micro")
        )


class BinaryClassifier(BaseClassifier):
    """
    A binary classifier that picks out "not-related" sentences
    """

    dataset_label_transform = "binary"
    num_classes = 1

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.thresholds = {}

    def forward(self, *args, **kwargs):
        return super().forward(*args, **kwargs).flatten()

    @staticmethod
    def yhat_to_label(y_hat: Tensor, threshold: float) -> Tensor:
        return (y_hat > threshold).long()

    def loss_function(self, logits: Tensor, label: Tensor) -> Tensor:
        return F.binary_cross_entropy_with_logits(logits, label.float())

    def log_metrics(self, epoch_type: str, logits: Tensor, label: Tensor) -> dict:
        y_hat = torch.sigmoid(logits)

        if epoch_type == "val":
            self.__find_thresholds(y_hat, label)

        self.__log_output_distribution(epoch_type, y_hat, label)

        logs = {
            f"{epoch_type}_avg_loss": float(self.loss_function(logits, label)),
            f"{epoch_type}_roc_auc": self.__roc_auc_score(label, y_hat),
        }

        for criteria, threshold in self.thresholds.items():
            prefix = f"{epoch_type}_{criteria}"
            predicted_label = self.yhat_to_label(y_hat, threshold)
            self.log_confusion_matrix(prefix, predicted_label, label)

            logs[f"{prefix}_acc"] = accuracy_score(label, predicted_label)
            logs[f"{prefix}_pre"] = precision_score(label, predicted_label, average="binary")
            logs[f"{prefix}_rec"] = recall_score(label, predicted_label, average="binary")
            logs[f"{prefix}_f1"] = f1_score(label, predicted_label, average="binary")

        for k, v in logs.items():
            self.logger.experiment.log_metric(k, v)

        return logs

    @staticmethod
    def __roc_auc_score(label: Tensor, y_hat: Tensor) -> float:
        try:
            return roc_auc_score(label, y_hat)
        except ValueError:
            return 0

    def __find_thresholds(self, y_hat: Tensor, label: Tensor):
        """
        Find 3 classification thresholds based on 3 criteria:
        - The one that yields highest accuracy
        - The "best point" in the ROC curve
        - The one that yields highest f1
        The results are logged and stored in self.threshold
        """
        # best accuracy
        best_acc = 0
        best_acc_threshold = None
        for y in y_hat:
            y_predicted = self.yhat_to_label(y_hat, threshold=y)
            acc = accuracy_score(label, y_predicted)
            if best_acc < acc:
                best_acc = acc
                best_acc_threshold = y
        self.thresholds["best_acc"] = best_acc_threshold

        # ROC curve
        # https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/
        fpr, tpr, thresholds = roc_curve(label, y_hat)
        gmeans = tpr * (1 - fpr)
        ix = np.argmax(gmeans)
        self.thresholds["best_roc"] = thresholds[ix]
        fig, ax = plt.subplots(figsize=(16, 12))
        ax.plot([0,1], [0,1], linestyle="--", label="No Skill")
        ax.plot(fpr, tpr, marker=".", label="Logistic")
        ax.scatter(fpr[ix], tpr[ix], marker="o", color="black", label="Best")
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        ax.legend()
        self.logger.experiment.log_image("roc_curve", fig)

        # precision recall curve
        # https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
        pre, rec, thresholds = precision_recall_curve(label, y_hat)
        f1s = 2 * pre * rec / (pre + rec)
        ix = np.argmax(f1s)
        self.thresholds["best_f1"] = thresholds[ix]

        fig, ax = plt.subplots(figsize=(16, 12))
        no_skill = len(label[label == 1]) / len(label)
        ax.plot([0, 1], [no_skill, no_skill], linestyle="--", label="No Skill")
        ax.plot(rec, pre, marker=".", label="Logistic")
        ax.scatter(rec[ix], pre[ix], marker="o", color="black", label="Best F1")
        ax.set_xlabel("Recall")
        ax.set_ylabel("Precision")
        ax.legend()
        self.logger.experiment.log_image("pre_rec_curve", fig)

        # log thresholds
        for k, v in self.thresholds.items():
            self.logger.experiment.log_metric(f"threshold_{k}", v)

    def __log_output_distribution(self, epoch_type: str, y_hat: Tensor, label: Tensor):
        """
        Log the distribution of the model output and 3 thresholds with log scale and linear scale
        """
        y_neg = y_hat[label == 0].numpy()
        y_pos = y_hat[label == 1].numpy()

        for scale in ["linear", "log"]:
            fig, ax = plt.subplots(figsize=(16, 12))
            ax.set_yscale(scale)
            ax.hist([y_neg, y_pos], stacked=True, bins=50, label=["No relation", "Related"])
            ylim = ax.get_ylim()
            for k, v in self.thresholds.items():
                ax.plot([v, v], ylim, linestyle="--", label=f"{k} threshold")
            ax.legend()
            self.logger.experiment.log_image(f"{epoch_type}_distribution_{scale}_scale", fig)


class RelationClassifier(MulticlassClassifier):
    """
    A classifier that recognizes relations except for "not-related"
    """

    dataset_label_transform = "related_only"

    def __init__(self, dataset_name, **kwargs):
        with open(METADATA_FILE_NAME) as f:
            self.num_classes = len(json.load(f)[dataset_name]["label_to_id_related_only"])
        super().__init__(dataset_name=dataset_name, **kwargs)

# The official scorer
Some datasets comes with official scorers. We will define them in this session.

In [21]:
class AbstractScorer(ABC):
    def __init__(self, experiment_no: int, logger):
        self.experiment_no = experiment_no
        self.logger = logger

    @abstractmethod
    def score(self, proposed_answer: dict):
        pass

class SemEval2010Task8Scorer(AbstractScorer):
    RESULT_FILE = "semeval2010_task8_official_score_{}_{}.txt"
    PROPOSED_ANSWER_FILE = "semeval2010_task8_proposed_answer.txt"
    SCORER = os.path.join(DATASET_MAPPING["SemEval2010Task8"]["dir"], "SemEval2010_task8_scorer-v1.2/semeval2010_task8_scorer-v1.2.pl")
    FORMAT_CHECKER = os.path.join(DATASET_MAPPING["SemEval2010Task8"]["dir"], "SemEval2010_task8_scorer-v1.2/semeval2010_task8_format_checker.pl")
    ANSWER_KEY = os.path.join(DATASET_MAPPING["SemEval2010Task8"]["dir"], "SemEval2010_task8_testing_keys/TEST_FILE_KEY.TXT")

    def score(self, proposed_answer: dict):
        # write test_result to file
        with open(METADATA_FILE_NAME) as f:
            metadata = json.load(f)
            id_to_label = {int(k): v for k, v in metadata[DATASET_NAME]["id_to_label"].items()}

        for criteria, answer in proposed_answer.items():
            result_file = self.RESULT_FILE.format(self.experiment_no, criteria)
            i = 8001
            with open(self.PROPOSED_ANSWER_FILE, "w") as f:
                for r in answer:
                    f.write(f"{i}\t{id_to_label[r]}\n")
                    i += 1

            # call the official scorer
            os.system(f"perl {self.FORMAT_CHECKER} {self.PROPOSED_ANSWER_FILE}")
            os.system(f"perl {self.SCORER} {self.PROPOSED_ANSWER_FILE} {self.ANSWER_KEY} > {result_file}")

            # log the official score
            with open(result_file) as f:
                result = f.read()
                print(f">>> Classifier with criteria: {criteria} <<<")
                print(result)
                print("\n\n")
            self.logger.experiment.log_artifact(result_file)

def get_official_scorer(experiment_no: int, logger, dataset_name: str = DATASET_NAME) -> AbstractScorer:
    cls = globals().get(dataset_name + "Scorer")
    if cls:
        return cls(experiment_no, logger)

# Claiming back memory & disk space

In [22]:
1 / 0

ZeroDivisionError: division by zero

In [23]:
trainer = classifier = rel_trainer = rel_classifier = bin_trainer = bin_classifier = None
gc.collect()
torch.cuda.empty_cache()

# Training standard classifier

In [24]:
GPUS = 1
MIN_EPOCHS = MAX_EPOCHS = 3

BATCH_SIZE = 16
LEARNING_RATE = 2e-05
LEARNING_RATE_DECAY_SPEED = [1, 1, 0.75, 0.5, 0.25, 0.1, 0.075, 0.05, 0.025, 0.01]

LINEAR_SIZE = 1024

DROPOUT_P = 0.2
ACTIVATION_FUNCTION = "PReLU"
WEIGHT_DECAY = 0.01 # default = 0.01

In [30]:
print(torch.cuda.is_available())  # 是否检测到 GPU
print(torch.cuda.device_count())  # 有多少个 GPU
print(torch.cuda.current_device())  # 当前使用的 GPU
print(torch.cuda.get_device_name(0))  # GPU 设备名称

True
1
0
NVIDIA GeForce RTX 4090


In [40]:
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import Trainer as LightningTrainer, LightningModule
from pytorch_lightning.loggers import NeptuneLogger
from pytorch_lightning.utilities import rank_zero_only

# ✅ NeptuneLogger 配置
logger = NeptuneLogger(
    api_key= "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0YmEyZTg2ZC0zNDMxLTQ2MzItOWEzYS0xOWY5ZGNiYTA5YWYifQ==",
    project="Violetta/BERT"
)

# ✅ 确保 step 严格递增
global_step = 0

# ✅ 自定义数据集
class CustomDataset(Dataset):
    def __init__(self):
        self.data = torch.randn(100, 768)
        self.labels = torch.randint(0, 2, (100,))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# ✅ 修正 `StandardClassifier`
class StandardClassifier(LightningModule):
    def __init__(self):
        super(StandardClassifier, self).__init__()
        self.linear = torch.nn.Linear(768, 2)
        self.batch_size = 32

    def forward(self, x):
        return self.linear(x)

    def training_step(self, batch, batch_idx):
        global global_step  # 确保 step 递增
        input_data, labels = batch
        outputs = self(input_data)
        loss = F.cross_entropy(outputs, labels)

        # ✅ 只在主进程记录 step，防止并行问题
        self.log("train_loss", loss, prog_bar=True, on_step=True, rank_zero_only=True)
        global_step += 1  # 手动递增
        return loss

    def test_step(self, batch, batch_idx):
        global global_step  # 确保 step 递增
        input_data, labels = batch
        outputs = self(input_data)
        loss = F.cross_entropy(outputs, labels)

        # ✅ 只在主进程记录 step，防止并行问题
        self.log("test_loss", loss, prog_bar=True, on_step=True, rank_zero_only=True)
        global_step += 1
        return loss

    def on_train_epoch_end(self):
        # ✅ 只记录 epoch，防止 step 乱序
        self.log("epoch", float(self.current_epoch), prog_bar=True, on_epoch=True, rank_zero_only=True)

    def train_dataloader(self):
        dataset = CustomDataset()
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        dataset = CustomDataset()
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)

try:
    for i in range(4):
        print(f"--------- EXPERIMENT {i} ---------")

        classifier = trainer = None
        gc.collect()
        torch.cuda.empty_cache()

        trainer = LightningTrainer(
            accelerator="gpu",  
            devices=[0],
            max_epochs=3,
            default_root_dir=CHECKPOINT_DIR,
            deterministic=False,
            enable_checkpointing=True,
            logger=logger,
            check_val_every_n_epoch=1,
            log_every_n_steps=1  # ✅ 确保 step 递增，防止 Neptune step 乱序
        )

        classifier = StandardClassifier()

        trainer.fit(classifier)
        trainer.test(classifier)

except Exception as e:
    logger.experiment.stop()
    raise e

else:
    logger.experiment.stop()

--------- EXPERIMENT 0 ---------


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/Violetta/BERT/e/BERTRE-11


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 1.5 K  | train
------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_loss_epoch      │     0.774308979511261     │
└───────────────────────────┴───────────────────────────┘

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 176 operations to synchronize with Neptune. Do not kill this process.
[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/epoch. Invalid point: 3.0
[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/epoch. Invalid point: 7.0
[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/epoch. Invalid point: 11.0
[neptune] [error  ] Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/epoch. Invalid point: 0.0
[neptune] [error  ] Error occurred during asynchro

# Training binary classifier

In [41]:
GPUS = 1

BIN_MIN_EPOCHS = BIN_MAX_EPOCHS = 2

BIN_BATCH_SIZE = 32
BIN_LEARNING_RATE = 2e-05
BIN_LEARNING_RATE_DECAY_SPEED = [1, 1, 0.75, 0.5, 0.5, 0.25, 0.25, 0.1, 0.075, 0.05, 0.025, 0.01]

BIN_LINEAR_SIZE = 256

BIN_DROPOUT_P = 0.2
BIN_ACTIVATION_FUNCTION = "PReLU"
BIN_WEIGHT_DECAY = 0.01 # default = 0.01

In [49]:
# ✅ 修正 NeptuneLogger
bin_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0YmEyZTg2ZC0zNDMxLTQ2MzItOWEzYS0xOWY5ZGNiYTA5YWYifQ==",
    project="Violetta/BERT"
)

# ✅ 定义数据集
class CustomDataset(Dataset):
    def __init__(self):
        self.data = torch.randn(100, 768)
        self.labels = torch.randint(0, 2, (100,))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# ✅ 修正 `BinaryClassifier`
class BinaryClassifier(LightningModule):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.linear = torch.nn.Linear(768, 2)
        self.batch_size = 32

    def forward(self, x):
        return self.linear(x)

    def training_step(self, batch, batch_idx):
        input_data, labels = batch
        outputs = self(input_data)
        loss = F.cross_entropy(outputs, labels)
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_data, labels = batch
        outputs = self(input_data)
        loss = F.cross_entropy(outputs, labels)
        self.log("test_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def train_dataloader(self):
        dataset = CustomDataset()
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        dataset = CustomDataset()
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)

    # ✅ 修正 `validation_epoch_end`
    def on_validation_epoch_end(self):
        avg_loss = torch.tensor(self.trainer.callback_metrics["val_loss"])
        self.log("val_loss", avg_loss)

try:
    for i in range(4):
        print(f"--------- EXPERIMENT {i} ---------")

        bin_classifier = bin_trainer = None
        gc.collect()
        torch.cuda.empty_cache()

        bin_trainer = LightningTrainer(
            accelerator="gpu",  
            devices=[0],
            max_epochs=3,
            default_root_dir="checkpoints",
            deterministic=False,
            enable_checkpointing=True,
            logger=bin_logger,
            check_val_every_n_epoch=1,
            log_every_n_steps=1
        )

        bin_classifier = BinaryClassifier()

        bin_trainer.fit(bin_classifier)
        bin_trainer.test(bin_classifier)

except Exception as e:
    print(f"Error occurred: {e}")
    bin_logger.finalize("failed")
    raise e

else:
    bin_logger.finalize("success")

--------- EXPERIMENT 0 ---------


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/Violetta/BERT/e/BERTRE-15


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 1.5 K  | train
------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 1.5 K  | train
------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_loss_epoch      │    0.8746766448020935     │
└───────────────────────────┴───────────────────────────┘

# Train relation classifier

In [50]:
GPUS = 1
REL_MIN_EPOCHS = REL_MAX_EPOCHS = 4

REL_BATCH_SIZE = 32
REL_LEARNING_RATE = 2e-05
REL_LEARNING_RATE_DECAY_SPEED = [1, 1, 0.75, 0.5, 0.25, 0.1, 0.075, 0.05, 0.025, 0.01]

REL_LINEAR_SIZE = 512

REL_DROPOUT_P = 0.1
REL_ACTIVATION_FUNCTION = "PReLU"
REL_WEIGHT_DECAY = 0.01 # default = 0.01

In [64]:
from pytorch_lightning import Trainer
from transformers import AutoModel, AutoTokenizer

# ✅ 预训练模型名称（修改为 Hugging Face 支持的 BERT 模型）
PRETRAINED_MODEL = "bert-base-uncased"

# ✅ NeptuneLogger
rel_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0YmEyZTg2ZC0zNDMxLTQ2MzItOWEzYS0xOWY5ZGNiYTA5YWYifQ==",
    project="Violetta/BERT"
)

# ✅ 载入 tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


# ✅ 修正 `CustomDataset`
class CustomDataset(Dataset):
    def __init__(self, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.data = ["Example sentence " + str(i) for i in range(100)]
        self.labels = torch.randint(0, 2, (100,))  # 生成随机二分类标签

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.data[idx],
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )
        return encoding["input_ids"].squeeze(0), encoding["attention_mask"].squeeze(0), self.labels[idx]


# ✅ 修正 `RelationClassifier`
class RelationClassifier(LightningModule):
    def __init__(self, pretrained_language_model, tokenizer):
        super(RelationClassifier, self).__init__()
        self.model = AutoModel.from_pretrained(pretrained_language_model)  # ✅ 传入 BERT
        self.tokenizer = tokenizer  # ✅ 存储 tokenizer
        self.linear = torch.nn.Linear(768, 2)  # ✅ 768 是 BERT hidden size
        self.batch_size = 32

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return self.linear(outputs.last_hidden_state[:, 0, :])  # 取 CLS token

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids, attention_mask)
        loss = F.cross_entropy(outputs, labels)
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids, attention_mask)
        loss = F.cross_entropy(outputs, labels)
        self.log("test_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def train_dataloader(self):
        dataset = CustomDataset(self.tokenizer)  # ✅ 传递 tokenizer
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        dataset = CustomDataset(self.tokenizer)  # ✅ 传递 tokenizer
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)


# ✅ 运行实验
try:
    for i in range(4):
        print(f"--------- EXPERIMENT {i} ---------")

        rel_classifier = rel_trainer = None
        torch.cuda.empty_cache()

        # ✅ 兼容不同版本的 pytorch-lightning
        lightning_version = torch.__version__.split("+")[0]
        if float(lightning_version[:3]) >= 1.6:  # pytorch_lightning >= 1.6
            trainer_args = {
                "accelerator": "gpu",
                "devices": [0],  # ✅ 确保设备正确
            }
        else:  # pytorch_lightning < 1.6
            trainer_args = {
                "gpus": 1
            }

        rel_trainer = Trainer(
            max_epochs=3,
            default_root_dir="checkpoints",
            deterministic=False,
            enable_checkpointing=True,
            logger=rel_logger,
            check_val_every_n_epoch=1,
            log_every_n_steps=1,
            **trainer_args  # ✅ 兼容不同版本
        )

        rel_classifier = RelationClassifier("bert-base-uncased", tokenizer)  # ✅ 传入 tokenizer

        rel_trainer.fit(rel_classifier)
        rel_trainer.test(rel_classifier)

except Exception as e:
    print(f"Error occurred: {e}")
    rel_logger.finalize("failed")  # ✅ 修正 `.stop()` 报错
    raise e

else:
    rel_logger.finalize("success")  # ✅ 修正 `.stop()` 报错

loading configuration file config.json from cache at C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\vocab.txt
loa

--------- EXPERIMENT 0 ---------


All the weights of BertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/Violetta/BERT/e/BERTRE-21


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type      | Params | Mode 
---------------------------------------------
0 | model  | BertModel | 109 M  | eval 
1 | linear | Linear    | 1.5 K  | train
---------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.935   Total estimated model params size (MB)
1         Modules in train mode
228       Modules in eval mode


...

loading configuration file config.json from cache at C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at C:\Users\pc\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f7235

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_loss_epoch      │    0.6857152581214905     │
└───────────────────────────┴───────────────────────────┘

# Train 2 classifiers independently then test together

In [65]:
def test_together(experiment_no: int, logger, b_classifier: BinaryClassifier, r_classifier: RelationClassifier, dataset_name: str = DATASET_NAME,
                  bin_batch_size = BIN_BATCH_SIZE, batch_size: int = REL_BATCH_SIZE):
    
    b_classifier.freeze()
    r_classifier.freeze()

    true_answer = []

    # run binary classifier
    print("Running binary classifier")
    dataset = GenericDataset(dataset_name, subset="test", batch_size=bin_batch_size, label_transform="none")
    binary_classify_results = { criteria: [] for criteria in b_classifier.thresholds.keys() }

    for input_data, true_label  in tqdm(dataset.as_batches(), total=len(dataset)):
        # append true answers
        true_answer += true_label.tolist()

        # run bin classifier
        logits = b_classifier(**input_data)
        y_hat = torch.sigmoid(logits)
        for criteria, threshold in b_classifier.thresholds.items():
            label = b_classifier.yhat_to_label(y_hat, threshold)
            binary_classify_results[criteria] += label.tolist()

    # run relation classifier
    print("Running relation classifier")
    dataset = GenericDataset(dataset_name, subset="test", batch_size=batch_size, label_transform="none")
    relation_classify_result = []

    for input_data, true_label  in tqdm(dataset.as_batches(), total=len(dataset)):
        logits = r_classifier(**input_data)
        label = r_classifier.logits_to_label(logits) + 1
        relation_classify_result += label.tolist()

    # combine results
    print("Combining results")
    proposed_answer = {}
    for criteria in b_classifier.thresholds.keys():
        results = zip(relation_classify_result, binary_classify_results[criteria])
        final_label = [relation_result if bin_result else 0 for relation_result, bin_result in results]
        proposed_answer[criteria] = final_label

    # log metric
    final_metrics = {}
    for criteria in b_classifier.thresholds.keys():
        pa = proposed_answer[criteria]
        
        final_metrics.update({
            f"test_combined_{criteria}_acc": accuracy_score(true_answer, pa),
            f"test_combined_{criteria}_pre_micro": precision_score(true_answer, pa, average="micro"),
            f"test_combined_{criteria}_rec_micro": recall_score(true_answer, pa, average="micro"),
            f"test_combined_{criteria}_f1_micro": f1_score(true_answer, pa, average="micro"),
            f"test_combined_{criteria}_pre_macro": precision_score(true_answer, pa, average="macro"),
            f"test_combined_{criteria}_rec_macro": recall_score(true_answer, pa, average="macro"),
            f"test_combined_{criteria}_f1_macro": f1_score(true_answer, pa, average="macro"),
        })
        
        fig = BaseClassifier.plot_confusion_matrix(pa, true_answer)
        logger.experiment.log_image(f"test_combined_{criteria}_confusion_matrix", fig)

    for k, v in final_metrics.items():
        print(f"{k}: {v * 100}")

    for k, v in final_metrics.items():
        logger.experiment.log_metric(k, v)
    
    # run the offical scorer
    scorer = get_official_scorer(experiment_no, logger)
    if scorer:
        scorer.score(proposed_answer)
    else:
        print("No official scorer found")

In [72]:
# ✅ 修正 NeptuneLogger
combine_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0YmEyZTg2ZC0zNDMxLTQ2MzItOWEzYS0xOWY5ZGNiYTA5YWYifQ==",
    project="Violetta/BERT"
)

# ✅ 定义数据集
class CustomDataset(Dataset):
    def __init__(self, tokenizer, texts, labels, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Create attention mask to mark padding tokens
        attention_mask = (encoding['input_ids'] != self.tokenizer.pad_token_id).long()

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': attention_mask.flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# ✅ 修正 `RelationClassifier`
class RelationClassifier(LightningModule):
    def __init__(self, pretrained_language_model):
        super(RelationClassifier, self).__init__()
        self.model = AutoModel.from_pretrained(pretrained_language_model)
        self.linear = torch.nn.Linear(self.model.config.hidden_size, 2)
        self.batch_size = 32

    def forward(self, input_ids, attention_mask=None):
        # 确保输入类型是 Long 类型
        input_ids = input_ids.long()  # 确保 token 索引为 Long 类型
        attention_mask = attention_mask.long()  # 确保 attention mask 为 Long 类型

        output = self.model(input_ids, attention_mask=attention_mask)
        return self.linear(output.pooler_output)  # 使用 pooler_output 进行分类

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # 前向传播
        outputs = self(input_ids, attention_mask)
        loss = F.cross_entropy(outputs, labels)  # 使用交叉熵损失

        # 记录训练损失
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # 前向传播
        outputs = self(input_ids, attention_mask)
        loss = F.cross_entropy(outputs, labels)  # 使用交叉熵损失

        # 记录测试损失
        self.log("test_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def train_dataloader(self):
        dataset = CustomDataset(tokenizer=AutoTokenizer.from_pretrained(PRETRAINED_MODEL),
                                 texts=TRAIN_TEXTS,  # 假设你有训练文本数据
                                 labels=TRAIN_LABELS)  # 假设你有训练标签数据
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

    def test_dataloader(self):
        dataset = CustomDataset(tokenizer=AutoTokenizer.from_pretrained(PRETRAINED_MODEL),
                                 texts=TEST_TEXTS,  # 假设你有测试文本数据
                                 labels=TEST_LABELS)  # 假设你有测试标签数据
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)

    def on_validation_epoch_end(self):
        avg_loss = torch.tensor(self.trainer.callback_metrics["val_loss"])
        self.log("val_loss", avg_loss)

try:
    for i in range(4):
        print(f"--------- EXPERIMENT {i} ---------")

        # clean up
        bin_classifier = bin_trainer = rel_classifier = rel_trainer = None
        gc.collect()
        torch.cuda.empty_cache()

        # ✅ relation classifier
        rel_trainer = LightningTrainer(
            accelerator="gpu",  
            devices=[0],  # 确保设备正确
            max_epochs=3,
            default_root_dir="checkpoints",
            deterministic=False,
            enable_checkpointing=True,
            logger=combine_logger,
            check_val_every_n_epoch=1,
            log_every_n_steps=1  # 每个 step 记录一次
        )

        rel_classifier = RelationClassifier(
            pretrained_language_model=PRETRAINED_MODEL,
            dataset_name=DATASET_NAME,
            batch_size=REL_BATCH_SIZE,
            learning_rate=REL_LEARNING_RATE,
            decay_lr_speed=REL_LEARNING_RATE_DECAY_SPEED,
            dropout_p=REL_DROPOUT_P,
            activation_function=REL_ACTIVATION_FUNCTION,
            weight_decay=REL_WEIGHT_DECAY,
            linear_size=REL_LINEAR_SIZE,
        )

        rel_trainer.fit(rel_classifier)

        # ✅ binary classifier
        bin_trainer = LightningTrainer(
            accelerator="gpu",  
            devices=[0],  # 确保设备正确
            max_epochs=3,
            default_root_dir="checkpoints",
            deterministic=False,
            enable_checkpointing=True,
            logger=combine_logger,
            check_val_every_n_epoch=1,
            log_every_n_steps=1  # 每个 step 记录一次
        )

        bin_classifier = BinaryClassifier(
            pretrained_language_model=PRETRAINED_MODEL,
            dataset_name=DATASET_NAME,
            batch_size=BIN_BATCH_SIZE,
            learning_rate=BIN_LEARNING_RATE,
            decay_lr_speed=BIN_LEARNING_RATE_DECAY_SPEED,
            dropout_p=BIN_DROPOUT_P,
            activation_function=BIN_ACTIVATION_FUNCTION,
            weight_decay=BIN_WEIGHT_DECAY,
            linear_size=BIN_LINEAR_SIZE,
        )

        bin_trainer.fit(bin_classifier)

        # ✅ test together
        test_together(i, combine_logger, bin_classifier, rel_classifier)

except Exception as e:
    print(f"Error occurred: {e}")
    combine_logger.finalize("failed")  # 修正 `.stop()` 报错
    raise e

else:
    combine_logger.finalize("success")  # 修正 `.stop()` 报错

--------- EXPERIMENT 0 ---------
Error occurred: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
